# Behaviour Cloning Project

## Import Needed Modules

In [1]:
import csv
import cv2
import os

import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import Sequential

Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.


## Generate Dataset from Raw Data

In [ ]:
def datasetGen():
    
    with open('simdata/driving_log.csv') as log_file:
        log_reader = csv.DictReader(log_file)
        count = 0
        X = np.empty([0, 160, 320, 3], dtype=float)
        y = np.empty([0, ], dtype=float)

        for row in log_reader:
            steering_offset = 0.4

            centerImage = mpimg.imread(row['center'].strip().replace('/home/era/Projects/Work/simdata', 'simdata'))
            flippedCenterImage = np.fliplr(centerImage)
            centerSteering = float(row['steering'])

            leftImage = mpimg.imread(row['left'].strip().replace('/home/era/Projects/Work/simdata', 'simdata'))
            flippedLeftImage = np.fliplr(leftImage)
            leftSteering = centerSteering + steering_offset

            rightImage = mpimg.imread(row['right'].strip().replace('/home/era/Projects/Work/simdata', 'simdata'))
            flippedRightImage = np.fliplr(rightImage)
            rightSteering = centerSteering - steering_offset

            X = np.append(X, 
                          np.array([centerImage, 
                                    flippedCenterImage, 
                                    leftImage, 
                                    flippedLeftImage, 
                                    rightImage, 
                                    flippedRightImage]), 
                         axis=0)
                        
            y = np.append(y, 
                          np.array([centerSteering, 
                                    -centerSteering, 
                                    leftSteering, 
                                    -leftSteering, 
                                    rightSteering, 
                                    -rightSteering]), 
                          axis=0)
    
    return X, y

In [ ]:
X, y = datasetGen()

## Network Architecture 

In [ ]:
def model():
    model = Sequential()

    model.add(layers.BatchNormalization(input_shape=(160, 320, 3), 
                                        axis=-1,
                                        momentum=0.99, 
                                        epsilon=0.001, 
                                        center=True, 
                                        scale=True, 
                                        beta_initializer='zeros', 
                                        gamma_initializer='ones', 
                                        moving_mean_initializer='zeros', 
                                        moving_variance_initializer='ones', 
                                        beta_regularizer=None, 
                                        gamma_regularizer=None, 
                                        beta_constraint=None, 
                                        gamma_constraint=None))

    model.add(layers.Conv2D(16, 
                            kernel_size=(5, 5), 
                            strides=(1, 1), 
                            activation='relu', 
                            padding='same'))

    model.add(layers.Conv2D(32, 
                            kernel_size=(5, 5), 
                            strides=(2, 2), 
                            activation='relu', 
                            padding='valid'))

    model.add(layers.AveragePooling2D(pool_size=(2, 2), 
                                      strides=(1, 1), 
                                      padding='valid'))

    model.add(layers.Conv2D(64, 
                            kernel_size=(3, 3), 
                            strides=(2, 2), 
                            activation='relu', 
                            padding='valid'))

    model.add(layers.Conv2D(64, 
                            kernel_size=(3, 3), 
                            strides=(2, 2), 
                            activation='relu', 
                            padding='valid'))

    model.add(layers.AveragePooling2D(pool_size=(2, 2), 
                                      strides=(1, 1), 
                                      padding='valid'))

    model.add(layers.Conv2D(128, 
                            kernel_size=(3, 3), 
                            strides=(1, 1), 
                            activation='relu', 
                            padding='valid'))

    model.add(layers.Conv2D(128, 
                            kernel_size=(3, 3), 
                            strides=(1, 1), 
                            activation='relu', 
                            padding='valid'))

    model.add(layers.Dropout(0.5))

    model.add(layers.Flatten())

    model.add(layers.Dense(4096, activation='linear'))

    model.add(layers.Dense(512, activation='linear'))

    model.add(layers.Dense(64, activation='linear'))

    model.add(layers.Dense(8, activation='linear'))

    model.add(layers.Dense(1, activation='linear'))

    model.compile(loss='mse', optimizer='adam')

    return model

In [ ]:
model = model()
model.summary()